In [1]:
# Caso seja necessário descomente e execute o comando abaixo. 
# Feito isso, será necessário reiniciar o notebook e executá-lo novamente
#pip install --upgrade category_encoders

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from category_encoders.one_hot import OneHotEncoder

## Carregando os datasets

In [ ]:
df1 = pd.read_csv('ACCOUNTS.csv')
df2 = pd.read_csv('DEMOGRAPHICS.csv')
df3 = pd.read_csv('LOANS.csv')

## Visão Geral

Compreender cada informação coletada é essencial para um possível modelo que será gerado. Nesse sentido, gera-se uma análise inicial de cada variável (ou coluna) foi coletada e justifica-se o porque dessa variável não será considerada no modelo ou até mesmo, qual tratamento de dados será utilizado nela.

In [ ]:
df1.info()

A função <strong>info()</strong> já mostra um resumo geral do comportamento dos dados, apesar de superficial é possível observar que os dados estão desbalanceados, ou seja, será necessário realizar uma análise exploratória individual de cada variável e posteriormente realizar um tratamento e limpeza de informações que não terão impacto no modelo de classificação que se almeja construir.

## - Análise Inicial de Dados 1 - 'accounts.csv'


In [ ]:
df1

In [ ]:
df1.corr() 

In [ ]:
df1[df1['CHECKING_BALANCE']=='NO_CHECKING']

In [ ]:
df1[df1['EXISTING_SAVINGS']=='UNKNOWN']

### 1) Com relação aos saldo da conta de cada cliente, qual a relação entre verificados e não verificados? 

In [ ]:
# Dividi-se o dataset em 2, onde a variável check é compreendida pelas informações numéricas.
no_check = len(df1[df1[u'CHECKING_BALANCE'] == 'NO_CHECKING'])
check = len(df1[df1[u'CHECKING_BALANCE'] != 'NO_CHECKING'])

new_df = {'NO_CHECKING': [no_check], 'CHECKING': [check]}

checking_balance = pd.DataFrame(data=new_df)

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(checking_balance) ,y=[checking_balance['CHECKING'].iloc[0],checking_balance['NO_CHECKING'].iloc[0]],hue=list(checking_balance))
plt.title("Relação entre saldos verificados e não verificados")
plt.show(fig)

### 2) Quais características estão presentes na coluna 'Histórico de Crédito'?

In [ ]:
df1['CREDIT_HISTORY'].unique()

### 3) Qual a relação entre economias conhecidas e desconhecidas dos clientes com o banco?

In [ ]:
# Dividi-se o dataset em 2, onde a variável check é compreendida pelas informações numéricas.
exis = len(df1[df1[u'EXISTING_SAVINGS'] != 'UNKNOWN'])
no_exis = len(df1[df1[u'EXISTING_SAVINGS'] == 'UNKNOWN'])

new_df2 = {'EXISTING_SAVINGS': [exis], 'NO_EXISTING_SAVINGS': [no_exis]}

check_savings = pd.DataFrame(data=new_df2)

In [ ]:
check_savings

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_savings) ,y=[check_savings['EXISTING_SAVINGS'].iloc[0],
                                            check_savings['NO_EXISTING_SAVINGS'].iloc[0]],
                                            hue=list(check_savings))
plt.title("Relação entre saldos verificados e não verificados")
plt.show(fig)

In [ ]:
df1[df1['EXISTING_CREDITS_COUNT'].isnull()]

## -  Análise Inicial de Dados 2 - 'demographics.csv'

In [ ]:
df2

In [ ]:
df2.info()

### Qual o sexo dos clientes?

In [ ]:
df2['SEX'].value_counts()

In [ ]:
df2['SEX'].fillna("NR", inplace = True)

In [ ]:
# Dividi-se o dataset em 2, onde a variável check é compreendida pelas informações numéricas.
sex_M = len(df2[df2[u'SEX'] == 'M'])
sex_F = len(df2[df2[u'SEX'] == 'F'])
sex_NR = len(df2[df2[u'SEX'] == 'NR']) # não respondido (NaN)

new_df2 = {'M': [sex_M], 'F': [sex_F]}

check_sex = pd.DataFrame(data=new_df2)

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_sex),
                  y=[check_sex['M'].iloc[0],
                     check_sex['F'].iloc[0]],
                     hue=list(check_sex))
plt.grid(None)
plt.title("Relação entre gênero")
plt.rcParams["figure.figsize"] = (4,6)
plt.show(fig)

Nesse caso, baseado nos dados optamos por apenas apresentar a relação entre o sexo Masculino e Feminino para dados bancários. Vale ressaltar que o índice percentual de pessoas que ou não responderam, ou que por alguma instabilidade do sistema não foi salva é de 0.4% no total (16 registros sem resposta). Mas isso não é tudo, é importante entender também qual amostra está a nossa disposição, no caso, essa amostra tem exatos 3936 dados.

### Qual a distribuição a cerca da idade dos nossos clientes?

Para auxiliar nessa tarefa, criamos alguns filtros para compreender melhor a qual faixa etária os clientes da BanTotal pertencem. 

In [ ]:
#df2['AGE'].value_counts()

In [ ]:
ax = sns.distplot(df2.AGE, kde = False, 
                  bins = [i*5 for(i) in range (15)])
ax.set_xlabel("Idade das pessoas", 
              fontsize = 20, 
              position=(0.5,0))

ax.set_ylabel("Quantidade de pessoas", 
              fontsize = 20, 
              position=(0.5,0.5))

ax.grid(None)

ax.set_title('Distribuição do número de pessoas', 
             fontsize = 20)

ax.figure.set_size_inches(12,8)

### Qual a distribuição que indica quantos dependentes nossos clientes têm?



In [ ]:
df2['DEPENDENTS'].value_counts()

In [ ]:
dep1 = len(df2[df2[u'DEPENDENTS'] == 1.0])
dep2 = len(df2[df2[u'DEPENDENTS'] == 2.0])

new_df3 = {'1.0': [dep1], '2.0': [dep2]}

check_dependence = pd.DataFrame(data=new_df3)
check_dependence

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_dependence),
                  y=[check_dependence['1.0'].iloc[0],
                     check_dependence['2.0'].iloc[0]],
                     hue=list(check_dependence))
plt.grid(None)
plt.title("Relação entre dependentes")
plt.rcParams["figure.figsize"] = (4,6)
plt.show(fig)

### O cliente tem telefone registrado no sistema?

0 - não

1 - sim


In [ ]:
df2['TELEPHONE'].value_counts()

In [ ]:
phone1 = len(df2[df2[u'TELEPHONE'] == 0.0])
phone2 = len(df2[df2[u'TELEPHONE'] == 1.0])

new_df4 = {'Sim': [phone1], 'Não': [phone2]}

check_phone = pd.DataFrame(data=new_df4)

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_phone),
                  y=[check_phone['Sim'].iloc[0],
                     check_phone['Não'].iloc[0]],
                     hue=list(check_phone))
plt.grid(None)
plt.title("O cliente tem telefone registrado no sistema?")
plt.rcParams["figure.figsize"] = (6,8)
plt.show(fig)

### O cliente é um trabalhador estrangeiro?



In [ ]:
df2['FOREIGN_WORKER'].value_counts()

In [ ]:
w1 = len(df2[df2[u'FOREIGN_WORKER'] == 0.0])
w2 = len(df2[df2[u'FOREIGN_WORKER'] == 1.0])

new_df4 = {'Sim': [w2], 'Não': [w1]}

check_worker = pd.DataFrame(data=new_df4)

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_worker),
                  y=[check_worker['Sim'].iloc[0],
                     check_worker['Não'].iloc[0]],
                     hue=list(check_worker))
plt.grid(None)
plt.title("O cliente tem telefone registrado no sistema?")
plt.rcParams["figure.figsize"] = (6,8)
plt.show(fig)

### Duração do Trabalho

In [ ]:
df2['EMPLOYMENT_DURATION'].value_counts()

In [ ]:
ax = sns.distplot(df2.EMPLOYMENT_DURATION, kde = False, 
                  bins = [i for(i) in range (17)])

ax.set_xlabel("Duração de Trabalho", 
              fontsize = 20, 
              position=(0.5,0))

ax.set_ylabel("Quantidade de pessoas", 
              fontsize = 20, 
              position=(0.5,0.5))

ax.grid(None)

ax.set_title('Distribuição da duração de trabalho', 
             fontsize = 20)

ax.figure.set_size_inches(12,8)

### Propriedade

In [ ]:
df2['PROPERTY'].unique()

### Habitação

In [ ]:
df2['HOUSING'].unique()

### Duração de Residência Atual

In [ ]:
df2['CURRENT_RESIDENCE_DURATION'].value_counts()

In [ ]:
crd1 = len(df2[df2[u'FOREIGN_WORKER'] == 1.0])
crd2 = len(df2[df2[u'CURRENT_RESIDENCE_DURATION'] == 2.0])
crd3 = len(df2[df2[u'CURRENT_RESIDENCE_DURATION'] == 3.0])
crd4 = len(df2[df2[u'CURRENT_RESIDENCE_DURATION'] == 4.0])
crd5 = len(df2[df2[u'CURRENT_RESIDENCE_DURATION'] == 5.0])
crd6 = len(df2[df2[u'CURRENT_RESIDENCE_DURATION'] == 6.0])

new_df5 = {'1': [crd1], '2': [crd2], '3': [crd3], '4': [crd4], '5': [crd5], '6': [crd6]}

check_crd = pd.DataFrame(data=new_df5)

In [ ]:
sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_crd),
                  y=[check_crd['1'].iloc[0],
                     check_crd['2'].iloc[0],
                     check_crd['3'].iloc[0],
                     check_crd['4'].iloc[0],
                     check_crd['5'].iloc[0],
                     check_crd['6'].iloc[0]],
                     hue=list(check_crd))
plt.grid(None)
plt.title("O cliente tem telefone registrado no sistema?")
plt.rcParams["figure.figsize"] = (12,12)
plt.show(fig)

#### Questão x: De acordo com os insights obtidos nas questões anteriores, quais serão as decisões de tratamento a serem adotadas nos dados?



## - Análise Inicial de Dados 3 - 'LOANS.csv'

In [ ]:
df3

Esse dataset é composto por informações amplamento correlacionadas a uma possível concessão de crédito para os clientes da BanTotal. Cada cliente é identificado por uma chave ID e também há outras variávéis como: Termo de Pagamento, Percentual de Parcelamento, Objetivo do Empréstimo, Montante do Empréstimo, Outros Empréstimos, Permitir. Mas fique atento, apenas a leitura superficial das colunas do conjunto de dados não é suficiente para tirar conclusões ou sair aplicando funções de limpeza de dados, é importante estudar individualmente cada variável e formular hipóteses de como remodelar essa variável para constribuir para o modelo que será construído. Um exemplo disso são as variáveis categóricas (não numéricas) contidas no conjunto de dados acima.

### Termo de Pagamento

In [ ]:
df3.PAYMENT_TERM

In [ ]:
ax = sns.distplot(df3.PAYMENT_TERM, kde = False)

ax.set_xlabel("Termo de Pagamento", 
              fontsize = 20, 
              position=(0.5,0))

ax.set_ylabel("Clientes", 
              fontsize = 20, 
              position=(0.5,0.5))

ax.grid(None)

ax.set_title('Distribuição da duração de trabalho', 
             fontsize = 20)

ax.figure.set_size_inches(20,18)

### Planos de Parcelamento

In [ ]:
df3['INSTALLMENT_PLANS'].unique()

### Percentual de Parcelamento

In [ ]:
ax = sns.distplot(df3.INSTALLMENT_PERCENT, kde = False)

ax.set_xlabel("Percentual de Parcelamento", 
              fontsize = 20, 
              position=(0.5,0))

ax.set_ylabel("Clientes", 
              fontsize = 20, 
              position=(0.5,0.5))

ax.grid(None)

ax.set_title('Distribuição de Percentual de Parcelamento', 
             fontsize = 20)

ax.figure.set_size_inches(5,5)

### Objetivo do Empréstimo

In [ ]:
aux = df3['LOAN_PURPOSE'].value_counts()
aux

In [ ]:
result = []
for i in range(len(df3['LOAN_PURPOSE'].value_counts())):
    result.append(aux[i])
result

In [ ]:
# Note que não conseguimos manipular as linhas apresentadas no objeto series gerado pela função .value_counts()
# Para solucionar, transformamos esse objeto series em um dataframe e listamos os seus índices
aux2 = pd.DataFrame(aux)
montantes = list(aux2.index)
montantes

In [ ]:
# Plota-se o gráfico entre os tipos de objetivo de empréstimo e quantidade de clientes
plt.rcParams["figure.figsize"] = (8,7)
plt.barh(montantes, result, color='blue')

### Montante do Empréstimo

In [ ]:
ax = sns.distplot(df3.LOAN_AMOUNT, kde = False)

ax.set_xlabel("Percentual de Parcelamento", 
              fontsize = 20, 
              position=(0.5,0))

ax.set_ylabel("Clientes", 
              fontsize = 20, 
              position=(0.5,0.5))

ax.grid(None)

ax.set_title('Distribuição do Montante do Empréstimo', 
             fontsize = 20)

ax.figure.set_size_inches(12,10)

### Outros Empréstimos Ativos

In [ ]:
aux = df3['OTHERS_ON_LOAN'].value_counts()
result = []
for i in range(len(df3['OTHERS_ON_LOAN'].value_counts())):
    result.append(aux[i])

aux2 = pd.DataFrame(aux)
others = list(aux2.index)

plt.rcParams["figure.figsize"] = (4,3)
plt.barh(others, result, color='blue')

# Permitir Empréstimo?

Essa é variável no qual pretende-se classificar com a entrada de novos clientes. De forma objetiva, ela só pode retornar dois possíveis valores: 0 caso o modelo indique que o empréstimo não será concedido, e 1 caso o empréstimo será concedido.

In [ ]:
yes = len(df3[df3[u'ALLOW'] == 1])
no = len(df3[df3[u'ALLOW'] == 0])

new_df = {'Permitir': [yes], 'Não Permitir': [no]}

check_allow = pd.DataFrame(data=new_df)

sns.set(style="darkgrid")

fig = sns.barplot(x=list(check_allow),
                  y=[check_allow['Permitir'].iloc[0],
                     check_allow['Não Permitir'].iloc[0]],
                     hue=list(check_allow))
plt.grid(None)
plt.title("Qual decisão empréstimo foi aprovado?")
plt.rcParams["figure.figsize"] = (6,8)
plt.show(fig)

# Visão Geral

Compreender cada variável é uma etapa importante no processo da resolução do problema, pois caso optemos por adicionar, tratar ou remover uma variável do modelo que será treinado e testado, é de suma importância entender o que essa variável representa e por qual razão foi tomada uma determinada decisão em relação a ela. Dessa maneira você receberá insights valiosos para a criação da sua narrativa para storytelling de dados.

Mas isso não é tudo, na próxima seção será construído o tratamento de dados ausentes, variáveis categóricas e uma análise exploratória juntando as bases de dados através do Pandas.

# Tratamento de Dados

O tratamento de dados pode ser construídos de 2 formas, a primeira trataria variáveis categóricas associadas a um id como 0 ou 1, seguindo a lógica de um "One Hot Encoder". Outra maneira, seria determinar índices associados a uma determinada variável categórica como, por exemplo: 0 - não informado, 1 - ruim, 2 - médio, 3 - bom.

## Tratamento de Dados 1 - Preparando a variável categórica 'histórico de crédito' para o modelo de classificação

In [ ]:
enc = OneHotEncoder(cols=['CREDIT_HISTORY'], use_cat_names=True)
enc.fit(df1)

In [ ]:
df1['CREDIT_HISTORY'].nunique()

In [ ]:
new_df1 = enc.transform(df1)
new_df1